In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Load data
df = pd.read_csv('../data/sales.csv')

In [3]:
# Parse dates
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

In [4]:
# Clean revenue - fill nulls with median
median_revenue = df['revenue'].median()
df['revenue_clean'] = df['revenue'].fillna(median_revenue)

In [5]:
# Calculate derived metrics
df['profit'] = df['revenue_clean'] - df['cost']
df['margin'] = df['profit'] / df['revenue_clean']
df['is_profitable'] = df['profit'] > 0

In [6]:
# Create category encoding
category_map = {cat: i for i, cat in enumerate(df['category'].unique())}
df['category_code'] = df['category'].map(category_map)

In [7]:
# Regional aggregation
region_stats = df.groupby('region').agg({
    'revenue_clean': ['sum', 'mean', 'std'],
    'profit': ['sum', 'mean'],
    'is_profitable': 'mean'
}).round(2)
region_stats.columns = ['_'.join(col) for col in region_stats.columns]

In [8]:
# Time series aggregation
monthly = df.groupby(['year', 'month']).agg({
    'revenue_clean': 'sum',
    'profit': 'sum',
    'id': 'count'
}).rename(columns={'id': 'num_sales'})

In [9]:
# Category performance
category_perf = df.groupby('category').agg({
    'revenue_clean': 'sum',
    'profit': 'sum',
    'margin': 'mean'
}).sort_values('profit', ascending=False)

In [10]:
# Final summary
summary = {
    'total_revenue': df['revenue_clean'].sum(),
    'total_profit': df['profit'].sum(),
    'avg_margin': df['margin'].mean(),
    'num_regions': df['region'].nunique(),
    'num_categories': df['category'].nunique(),
    'date_range': (df['date'].min().strftime('%Y-%m-%d'), df['date'].max().strftime('%Y-%m-%d'))
}
summary

{'total_revenue': 334902.5,
 'total_profit': 117215.88,
 'avg_margin': 0.35,
 'num_regions': 4,
 'num_categories': 3,
 'date_range': ('2023-01-03', '2024-02-05')}